## 캐싱(Caching)

LangChain은 LLM을 위한 선택적 캐싱 레이어를 제공합니다.

이는 두 가지 이유로 유용합니다.

- 동일한 완료를 여러 번 요청하는 경우 LLM 공급자에 대한 **API 호출 횟수를 줄여 비용을 절감**할 수 있습니다.
- LLM 제공업체에 대한 **API 호출 횟수를 줄여 애플리케이션의 속도를 높일 수** 있습니다.

In [2]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [3]:
# Bedrock 설정 및 Util 함수 설정
from langchain_aws import ChatBedrockConverse
import os
from Util.stream_utils import print_stream_content, get_stream_content

# 객체 생성
llm = ChatBedrockConverse(
    aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
    aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
    model="apac.anthropic.claude-sonnet-4-20250514-v1:0",
    region_name="ap-northeast-2",
)

In [4]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH04-Models")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH04-Models


모델과 프롬프트를 생성합니다


In [5]:
from langchain_core.prompts import PromptTemplate

# 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template("{country} 에 대해서 200자 내외로 요약해줘")

# 체인을 생성합니다.
chain = prompt | llm

In [6]:
%%time 
response = chain.invoke({"country": "한국"})
print(response.content)

한국(대한민국)은 동아시아 한반도 남부에 위치한 국가입니다. 수도는 서울이며, 인구는 약 5,200만 명입니다. 1948년 건국 후 급속한 경제성장을 이뤄 선진국 반열에 올랐으며, 삼성, LG 등 글로벌 기업들이 있습니다. K-pop, K-드라마로 대표되는 한류가 전 세계적으로 인기를 끌고 있고, 김치, 불고기 등의 한식문화도 널리 알려져 있습니다. 민주주의 국가로서 높은 교육열과 IT 강국으로도 유명합니다.
CPU times: total: 312 ms
Wall time: 4.6 s


## InMemoryCache

인메모리 캐시를 사용하여 동일 질문에 대한 답변을 저장하고, 캐시에 저장된 답변을 반환합니다.

In [7]:
%%time
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache

# 인메모리 캐시를 사용합니다.
set_llm_cache(InMemoryCache())

# 체인을 실행합니다.
response = chain.invoke({"country": "한국"})
print(response.content)

한국(대한민국)은 동아시아 한반도 남부에 위치한 국가로, 인구 약 5,200만 명의 단일민족 국가입니다. 1948년 건국 후 6.25전쟁을 겪었으며, 1960년대부터 급속한 경제성장을 이뤄 선진국 대열에 합류했습니다. 반도체, 자동차, 조선업이 주력산업이며, K-pop, K-드라마 등 한류문화가 전 세계적으로 인기를 얻고 있습니다. 수도는 서울이고, 민주공화제 국가입니다.
CPU times: total: 234 ms
Wall time: 4.4 s


In [8]:
%%time
# 체인을 실행합니다.
response = chain.invoke({"country": "한국"})
print(response.content)

한국(대한민국)은 동아시아 한반도 남부에 위치한 국가로, 인구 약 5,200만 명의 단일민족 국가입니다. 1948년 건국 후 6.25전쟁을 겪었으며, 1960년대부터 급속한 경제성장을 이뤄 선진국 대열에 합류했습니다. 반도체, 자동차, 조선업이 주력산업이며, K-pop, K-드라마 등 한류문화가 전 세계적으로 인기를 얻고 있습니다. 수도는 서울이고, 민주공화제 국가입니다.
CPU times: total: 0 ns
Wall time: 6.52 ms


## SQLite Cache


In [9]:
from langchain_community.cache import SQLiteCache
from langchain_core.globals import set_llm_cache
import os

# 캐시 디렉토리를 생성합니다.
if not os.path.exists("cache"):
    os.makedirs("cache")

# SQLiteCache를 사용합니다.
set_llm_cache(SQLiteCache(database_path="cache/llm_cache.db"))

In [15]:
%%time 
# 체인을 실행합니다.
response = chain.invoke({"country": "한국"})
print(response.content)

한국은 동아시아에 위치한 반도 국가로, 현재 대한민국(남한)과 조선민주주의인민공화국(북한)으로 분단되어 있습니다. 수천 년의 독특한 문화와 역사를 가지고 있으며, 한글이라는 고유 문자를 사용합니다. 대한민국은 1960년대 이후 급속한 경제성장을 이루어 선진국 반열에 올랐으며, K-pop, 드라마, 영화 등 한류 문화가 전 세계적으로 인기를 얻고 있습니다. 수도는 서울이고, 인구는 약 5천만 명입니다.
CPU times: total: 0 ns
Wall time: 7.11 ms
